Author: Gickel OKABI

Python developer

mail : i.gickelokabi@gmail.com

.

Project :

Hi,

I want to get the contacts (email adress, name, phone number if any, clinic, address ) of hundreds of professionals who are on different specialized platforms:
-  https://www.ahvma.org/find-a-holistic-veterinarian/#!directory/map/ord=lnm)  or
- https://online.acvs.org/acvsssa/rflssareferral.query_page?P_VENDOR_TY=VETS

Their email addresses are not directly on their website.

It must be delivered in an excel file and email addresses must be valid.

Need to be done by the end of next week.

Best.

.

In [68]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [79]:
headers = {
    'Referer': 'https://www.kijiji.ca/',
    'Sec-Ch-Ua': 'v="Chromium";v="114", " Not A;Brand";v="8", "Google Chrome";v="114"',

    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': "Windows",
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    }
    

In [69]:
# This function takes as parameters the start page and the last page to scrape and returns all the pages
def get_page_lint(starting_age, ending_page):
    
    all_page_link = []
    
    for i in range(starting_age, 1+ending_page):
        link = f"https://online.acvs.org/acvsssa/rflssareferral.result_page?p_cust_id=&p_vendor_ty=VETS&p_query_ty=QUERY_PAGE&p_session_serno=1766006&p_search_serno=2080468&p_search_rule=&p_keyword=&p_city_nm=&p_state_cd=&p_zip=&p_country_cd=&p_search_latitude=&p_search_longitude=&p_email=&p_distance=500&p_page_size=100&p_page_num={i}&p_portfolio_only=&p_first_nm=&p_last_nm=&p_total_page=20"
        all_page_link.append(link)
        
    return all_page_link    

In [70]:
all_page_link = get_page_lint(1,2)

In [74]:
def data_collecte(all_car_link, valide_proxies):
    #Loop on the list of proxies
    proxy_cycle = cycle(valide_proxies)
    names = []
    mails = []
    phones = []
    clinics = []
    clinics_adress= []
    for page_link in all_page_link:
        proxy = next(proxy_cycle)
        response =  requests.get(url, proxies=proxy, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        veterinaries = soup.find_all('div', class_="aaRflResultWrapper")

        for veterinary in veterinaries:
            try:
                name = veterinary.find('li', class_='aaRflResultDrillDown').text.split(',')[0]
            except:
                name = ""
            try:    
                mail = veterinary.find('li', class_="aaRflResultEmail").text
            except:
                mail =""
            try:    
                phone = veterinary.find('li', class_="aaRflResultPhone").text
            except:
                phone =""
            try:    
                clinic = veterinary.find_all('li', class_="aaRflResultAddr")
                for c in clinic:
                    chaine = str(c).split('<br/>')
   
                    if(len(chaine) > 3 and len(chaine) < 6 ):
                        clinic_name = re.sub(r'<.*?>','', chaine[0])
                        clinic_addr = re.sub(r'<.*?>','', " ".join(chaine[1:]))   
                    elif(len(chaine)  >= 6):
                        clinic_name = re.sub(r'<.*?>','', " ".join(chaine[0:2]))
                        clinic_addr = re.sub(r'<.*?>','', " ".join(chaine[2:]))  
                    else:
                        clinic_name = 'independent'
                        clinic_addr = re.sub(r'<.*?>','', " ".join(chaine[0:]))   
            except:
                clinic_name = ""
                clinic_addr = ""  
                
            # Save items to list    
            names.append(name)
            mails.append(mail)
            phones.append(phone)
            clinics.append(clinic_name)
            clinics_adress.append(clinic_addr)   
    
    #creation of the data frame 
    dataframe = pd.DataFrame({
                'names' : names,
                'mails':mails,
                'phones' : phones,
                'clinics' :clinics,
                'clinics_adress' : clinics_adress,
        
            })  
    
    return dataframe       

.

In [75]:
dataframe = data_collecte(all_page_link)

In [78]:
#display the first 10 lines
dataframe.head(10)

,names,mails,phones,clinics,clinics_adress
0,Steven B. Abel,abelsb@sbcglobal.net,Phone:(630) 896-8541,VCA Aurora Animal Hospital,"2600 W Galena Blvd Aurora, IL 60506-9013"
1,LeeAnn W. Ablin,lablin@ameritech.net,Phone:(954) 437-9630,VCA Aurora Animal Hospital,"2600 W Galena Blvd Aurora, IL 60506-9013"
2,Gustavo A. Abuja,gustavo.abuja@gmail.com,Phone:(845) 532-5913,Rhinebeck Equine L.L.P,"26 Losee Ln Rhinebeck, NY 12572-2150"
3,Anthony E. Acquaviva,,Phone:(203) 929-8600,VCA Shoreline Veterinary Referral &amp; Emerge...,"895 Bridgeport Ave Shelton, CT 06484-4621"
4,Henry S. Adair III,sadair@utk.edu,Phone:(865) 974-5703,University of Tennessee,Dept Large Animal Clinical Science 2407 River ...
5,Emma N. Adam,emma@emmaadam.com,Phone:(484) 880-9470,Emma Adam LLC,"PO Box 12243 Lexington, KY 40582-2243"
6,David Adam-Castrillo,buckscountyequine@comcast.net,Phone:(215) 297-8470,"Bucks County Equine, LLC","3161 Creamery Rd New Hope, PA 18938-5636"
7,Krista Adamovich,,Phone:(512) 892-9038,Central Texas Veterinary Speciality Hospital,"4434 Frontier Trl Austin, TX 78745-1514"
8,Aric R. Adams,aadams@emcocala.com,Phone:(352) 479-0480,Equine Medical Center of Ocala,"7107 W Highway 326 Ocala, FL 34482-1244"
9,M. Norris Adams,mnadams@vt.edu,Phone:(703) 771-6800,Equine Medical Center of Ocala,"7107 W Highway 326 Ocala, FL 34482-1244"


.

In [ ]:
# Export to excel format 
dataframe.to_excel("donnees.csv", index=False)

.

Thanks for looking through the notebook.

If you have a data collection project, do not hesitate to entrust it to me. I will provide you with both the code and the collected dataset.

Gickel OKABI

mail : i.gickelokabi@gmail.com